In [1]:
! pip install -q sympy

In [2]:
import secrets
import math
from typing import List,Tuple

import sympy


In [3]:
NUM_PLAYERS: int = 3
CORRECT_PARAM_BIPRIME: int = 40

KEY_LENGTH = 128
PRIME_LENGTH = KEY_LENGTH // 2
PRIME_THRESHOLD = 2000
if PRIME_LENGTH < math.log(PRIME_THRESHOLD):
    PRIME_THRESHOLD = 1
PRIME_LIST = list(sympy.primerange(3, PRIME_THRESHOLD + 1))
"""
The additive shares of $p$ and $q$ are reshard to a Shamir secret sharing modulo a large prime P.
To assure P > N, we choose a prime P of at least 2*(prime_length + log_2 num_of_players) bits.
"""
SHAMIR_LENGTH = 2 * (PRIME_LENGTH + math.ceil((math.log2(NUM_PLAYERS))))
PRIME = sympy.nextprime(2 ** SHAMIR_LENGTH)

In [4]:
def extended_euclidean(num_a: int, num_b: int) -> Tuple[int, int, int]:
    """
    Perform the extended euclidean algorithm on the input numbers.
    The method returns gcd, x, y, such that a*x + b*y = gcd.

    :param num_a: First number a.
    :param num_b: Second number b.
    :return: Tuple containing gcd, x, and y, such that  a*x + b*y = gcd.
    """
    # a*x + b*y = gcd
    x_old, x_cur, y_old, y_cur = 0, 1, 1, 0
    while num_a != 0:
        quotient, num_b, num_a = num_b // num_a, num_a, num_b % num_a
        y_old, y_cur = y_cur, y_old - quotient * y_cur
        x_old, x_cur = x_cur, x_old - quotient * x_cur
    return num_b, x_old, y_old

def mod_inv(value: int, modulus: int) -> int:
    """
    Compute the inverse of a number, given the modulus of the group.
    Note that the inverse might not exist.

    :param value: The number to be inverted.
    :param modulus: The group modulus.
    :raise ZeroDivisionError: Raised when the inverse of the value does not exist.
    :return: The inverse of a under the modulus.
    """
    value %= modulus
    # else
    gcd_, inverse, _ = extended_euclidean(value, modulus)
    if gcd_ != 1:
        raise ZeroDivisionError(f"Inverse of {value} mod {modulus} does not exist.")
    return inverse

def pow_mod(base: int, exponent: int, modulus: int) -> int:
    """
    Compute base**exponent % modulus.

    :param base: base
    :param exponent: exponent
    :param modulus: modulus
    :return: base**exponent % modulus
    """
    if exponent < 0:
        return pow(mod_inv(base, modulus), -exponent, modulus)
    # else
    return pow(base, exponent, modulus)

In [5]:
def small_prime_divisors_test(prime_list: List[int], modulus: int) -> bool:
    """
    Function to test $N$ for small prime divisors
    :param prime_list: list of prime numbers
    :param modulus: modulus $N$
    :return: true if $N$ has small divisors and false otherwise
    """
    for prime in prime_list:
        if modulus % prime == 0:
            return True
    return False

In [6]:
def mul_v_and_check(v_values: List[int], modulus: int) -> bool:
    product = 1
    for idx, v in enumerate(v_values):
        if idx != 0:
            product *= v
    value1 = v_values[0]
    is_biprime = ((value1 % modulus) == (product % modulus)) or (
            (value1 % modulus) == (-product % modulus)
    )

    return is_biprime

def biprime_test(modulus:int, p_additive_shares: List[int], q_additive_shares: List[int]) -> bool:
    """
     Function to test for biprimality of $N$

    :param modulus: modulus $N$
    :param p_additive_shares: additive shares of $p$
    :param q_additive_shares: additive shares of $q$
    :return: true if $N$ is biprime and false otherwise
    """
    counter = 0
    while counter < CORRECT_PARAM_BIPRIME:
        test_values = []
        for index in range(0, NUM_PLAYERS):
            test_value_index = secrets.randbelow(modulus)
            test_values.append(test_value_index)
        test_value =  sum(test_values) % modulus

        if sympy.jacobi_symbol(test_value, modulus) == 1:
            v_values = []

            for index in range(0, NUM_PLAYERS):
                if index == 0:
                    v_value = int(
                        pow_mod(
                            test_value,
                            (
                                    modulus
                                    - p_additive_shares[index]
                                    - q_additive_shares[index]
                                    + 1
                            )
                            // 4,
                            modulus,
                        )
                    )
                else:
                    v_value = int(
                        pow_mod(
                            test_value,
                            (p_additive_shares[index] + q_additive_shares[index]) // 4,
                            modulus,
                        )
                    )
                v_values.append(v_value)

            if mul_v_and_check(v_values=v_values,modulus=modulus):
                counter += 1
            else:
                return False
    return True

In [7]:
def run_protocol():

    modulus: int = 0
    bip: bool = False
    bip_error_counter: int = 0
    sp_error_counter: int = 0
    while not bip:
        p_additive_shares: List[int] = []
        q_additive_shares: List[int] = []
        for index in range(0, NUM_PLAYERS):
            if index == 0:
                mod4 = 3
            else:
                mod4 = 0
            # Generate a prime $p$ of length $prime_length$ bits
            p_random_number = secrets.randbits(PRIME_LENGTH - 3) << 2
            p_additive_share: int = 2 ** (PRIME_LENGTH - 1) + p_random_number + mod4
            p_additive_shares.append(p_additive_share)

            # Generate a prime $q$ of length $prime_length$ bits
            q_random_number = secrets.randbits(PRIME_LENGTH - 3) << 2
            q_additive_share: int = 2 ** (PRIME_LENGTH - 1) + q_random_number + mod4
            q_additive_shares.append(q_additive_share)
        

        modulus = ((sum(p_additive_shares) % PRIME) * (sum(q_additive_shares) % PRIME)) % PRIME


        if not small_prime_divisors_test(PRIME_LIST, modulus):
            bip = biprime_test(modulus=modulus,p_additive_shares=p_additive_shares,q_additive_shares=q_additive_shares)
            if not bip:
                bip_error_counter += 1
        else:
            sp_error_counter += 1
    return bip_error_counter, sp_error_counter, p_additive_shares, q_additive_shares, modulus

In [8]:
bip_error_counter, sp_error_counter, p_additive_shares, q_additive_shares, modulus = run_protocol()

In [9]:
p = sum(p_additive_shares) % PRIME
q = sum(q_additive_shares) % PRIME
assert sympy.isprime(p), "p is not a prime number"
assert sympy.isprime(q), "q is not a prime number"

In [10]:
print(f"biprime error counter: {bip_error_counter} - small prime divisors error counter: {sp_error_counter}")
print(f"p: {p} - q: {q}")
print(f"modulus: {modulus}")

biprime error counter: 0 - small prime divisors error counter: 17
p: 44349117179578914743 - q: 40503326637012665519
modulus: 1796286779187634674539975092447676846617
